In [1]:
# general
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from collections import Counter
import collections
from pandas import DataFrame
from scipy.cluster.hierarchy import linkage, dendrogram

# sk learn
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.metrics.pairwise import linear_kernel
import matplotlib.pyplot as plt 

In [2]:
# traveler profile
data3 = pd.read_excel('data/users_full_7034.xlsx')
# traveler and reviews
data4 = pd.read_excel('data/reviews_32618_for_1098_users_with_location.xlsx')

In [3]:
null_age = data3.ageRange.isnull()
null_gender = data3.gender.isnull()
null_style = data3.travelStyle.isnull()




In [4]:
filtered_data3 = data3[data3.totalPoints > 1000][~null_age][~null_gender][~null_style]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [5]:
filtered_data3 = filtered_data3[['username']]

In [6]:
filtered_data4 = data4[['id', 'username', 'type', 'title', 'text', 'rating', 'taObjectCity']]


In [7]:
attraction_only = filtered_data4.type == 'Attractions'
filtered_data4 = filtered_data4[attraction_only]

In [9]:
span_mask1 = (filtered_data4.username == 'AnaS1')
span_mask2 = (filtered_data4.username == 'DaniLK')
span_mask3 = (filtered_data4.username == 'Aprile_24')
non_city_mask = (filtered_data4.taObjectCity == 'California')

In [10]:
filtered_data4 = filtered_data4[~span_mask1][~span_mask2][~non_city_mask][~span_mask3]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [11]:
data3_merge_4 = pd.merge(filtered_data4, filtered_data3, on=['username'])


In [12]:
# cities with more than 4 reviews
popular_city = []
for item, value in Counter(data3_merge_4.taObjectCity).items():
    if value > 7:
        popular_city.append(item)
print(len(popular_city))

56


In [13]:
final_df = data3_merge_4[data3_merge_4.taObjectCity.isin(popular_city)]


In [14]:
final_df.head()

,id,username,type,title,text,rating,taObjectCity
2,362,19Cam,Attractions,See it before its gone,The box is only a temporary exhibit while cons...,4,Berlin
3,363,19Cam,Attractions,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",3,Berlin
4,378,19Cam,Attractions,Fantastic,"After reading all the hype about this show, I ...",5,London
5,414,19Cam,Attractions,Tranquility in a busy city,Sited within the main botanical gardens area o...,4,Kuala Lumpur
18,778,1oldseagull,Attractions,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",5,Chattanooga


In [15]:
final_df.tail(5)

,id,username,type,title,text,rating,taObjectCity
2576,32510,FamilyTravelJunkkie,Attractions,If you travel at the right time there are no c...,Disneyland is great and so is California Adven...,4,Anaheim
2581,32516,FamilyTravelJunkkie,Attractions,If you are a cricket fan,If you are a cricket fan Kensington Oval is de...,4,Bridgetown
2591,32580,Farnborough,Attractions,A bit of a joke really,This is not actually the original or genuine C...,1,Berlin
2594,32612,Farnborough,Attractions,Not worth visiting,"As cat lovers, when we saw this mentioned in o...",1,Amsterdam
2595,32614,Farnborough,Attractions,Didn&acute;t live up to expectations,This didnвЂ™t really live up to expectations f...,2,Amsterdam


In [16]:
final_df.shape

(963, 7)

# clustering functions

In [17]:
cluster_input_df = final_df.copy()
cluster_input_df = cluster_input_df[['title', 'text', 'taObjectCity']]

In [18]:
cluster_input_df.head()

,title,text,taObjectCity
2,See it before its gone,The box is only a temporary exhibit while cons...,Berlin
3,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",Berlin
4,Fantastic,"After reading all the hype about this show, I ...",London
5,Tranquility in a busy city,Sited within the main botanical gardens area o...,Kuala Lumpur
18,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",Chattanooga


In [19]:
final_df.head()

,id,username,type,title,text,rating,taObjectCity
2,362,19Cam,Attractions,See it before its gone,The box is only a temporary exhibit while cons...,4,Berlin
3,363,19Cam,Attractions,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",3,Berlin
4,378,19Cam,Attractions,Fantastic,"After reading all the hype about this show, I ...",5,London
5,414,19Cam,Attractions,Tranquility in a busy city,Sited within the main botanical gardens area o...,4,Kuala Lumpur
18,778,1oldseagull,Attractions,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",5,Chattanooga


In [20]:
# span_mask1 = (final_df.username == 'AnaS1')
# span_mask2 = (final_df.username == 'DaniLK')
# span_mask3 = (final_df.username == 'Aprile_24')
# non_city_mask = (final_df.taObjectCity == 'California')


In [21]:
# final_df = final_df[~span_mask1][~span_mask2][~non_city_mask][~span_mask3]

In [22]:
final_df.shape

(963, 7)

In [23]:
# print(final_df.text.values[0])
# print(final_df.text.values[1])

In [24]:
df_title_comb = final_df.groupby(['taObjectCity']).apply(lambda x: ' '.join(x.title)).reset_index()
df_text_comb = final_df.groupby(['taObjectCity']).apply(lambda x: ' '.join(x.text)).reset_index()

In [25]:
df_text_comb.head()

,taObjectCity,0
0,Amsterdam,We found Van Gogh&acute;s story so interesting...
1,Anaheim,Disney has taken time to build the new cars la...
2,Bangkok,Every Bangkok visit should include the Grand p...
3,Barcelona,"The gardens are very pleasant, but nothing out..."
4,Bergen,A hike that will leave you tired but smiling. ...


In [26]:
cluster_input_df = df_title_comb.merge(df_text_comb, on= 'taObjectCity')

In [27]:
cluster_input_df.head()

,taObjectCity,0_x,0_y
0,Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
1,Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
2,Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
3,Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
4,Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [28]:
cluster_input_df.columns = ['taObjectCity','title','text']
cluster_input_df.head()

,taObjectCity,title,text
0,Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
1,Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
2,Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
3,Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
4,Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [29]:
cluster_input_df.set_index(['taObjectCity'], drop=True, inplace=True)

In [30]:
cluster_input_df.head()

,title,text
taObjectCity,,
Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [642]:


def cluster_texts(texts, clusters=3):
    """ 
    Transform texts to Tf-Idf coordinates and cluster texts using K-Means 
    """
    #my_additional_stop_words = ['acute', 'good', 'great', 'really', 'just', 'nice', 'like', 'day']
    my_additional_stop_words = ['acute', 'good', 'great', 'really', 'just', 'nice', 
                                'like', 'day', 'beautiful', 'visit', 'time', 'don',
                                'did', 'place', 'didn', 'did', 'tour', 'sydney','pm',
                                'lot', '00', 'inside', 'istanbul', 'doesn','going',
                                'right', '15']
    stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
    
    
    vectorizer = TfidfVectorizer(stop_words= stop_words,
                                 max_features = 400,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    vectors = tfidf_model.toarray()
    cols = vectorizer.get_feature_names()

    return (vectors, cols)


def cluster(vectors, cols, texts):
    """
    Cluster vecotirzed reviews and create k books a data frame relating the
    k label to the book id
    """
    kmeans = KMeans(n_clusters=5, random_state=2929292).fit(vectors)
    k_books = pd.DataFrame(list(zip(list(kmeans.labels_),
                                list(texts.index))),
                                columns=['cluster_k', 'city_index'])
    
    ''' added code to print centriod vocab - Print the top n words from all centroids vocab
    '''
    n = 20
    centroids = kmeans.cluster_centers_
    for ind, c in enumerate(centroids):
        print(ind)
        indices = c.argsort()[-1:-n-1:-1]
        print([cols[i] for i in indices])
        print("=="*20)
    
    return k_books

In [643]:
#reviews = cluster_input_df.title + ' ' + cluster_input_df.text
reviews = cluster_input_df.text

#reviews = final_df.title + ' ' + final_df.text
content = [i for i in reviews]


In [644]:
vector, cols = cluster_texts(content, 5)
cluster_df = cluster(vector, cols, reviews)

0
['museum', 'history', 'interesting', 'guide', 'building', 'hours', 'walk', 'worth', 'city', 'people', 'way', 'church', 'lovely', 'took', 'art', 'went', 'tickets', 'main', 'floor', 'little']
1
['worth', 'experience', 'fun', 'food', 'free', 'people', 'bar', 'trip', 'view', 'way', 'park', 'minutes', 'boat', 'went', 'bay', 'drinks', 'hotel', 'san', 'city', 'need']
2
['trail', 'falls', 'hike', 'road', 'memorial', 'water', 'area', 'views', 'bear', 'beach', 'walk', 'easy', 'morning', 'make', 'minutes', 'way', 'ride', 'view', 'parking', 'boat']
3
['town', 'worth', 'train', 'old', 'gardens', 'central', 'free', 'busy', 'walk', 'river', 'building', 'store', 'cars', 'area', 'best', 'friendly', 'lovely', 'views', 'bridge', 'walking']
4
['park', 'disney', 'parking', 'rides', 'love', 'food', 'kids', 'ride', 'movie', 'cars', 'pass', 'house', 'early', 'best', 'lots', 'fun', 'shows', 'husband', 'hour', 'area']


#593043848


0
['visit', 'beautiful', 'inside', 'place', 'tour', 'worth', 'building', 'museum', 'outside', 'impressive', 'interesting', 'lovely', 'time', 'barcelona', 'city', 'walk', 'excellent', 'cathedral', 'old', 'history']
========================================
1
['museum', 'ship', 'gate', 'tiki', 'temple', 'museums', 'story', 'beautiful', 'small', 'main', 'christmas', 'snow', 'time', 'visit', 'miss', 'pretty', 'minute', 'walk', '15', 'crowd']
========================================
2
['tour', 'time', 'worth', 'museum', 'place', 'visit', 'fun', 'history', 'guide', 'people', 'interesting', 'experience', 'lot', 'building', 'way', 'went', 'did', 'park', 'little', 'food']
========================================
3
['tour', 'train', 'free', 'place', 'visit', 'park', 'food', 'walk', 'fun', 'time', 'view', 'worth', 'hk', 'experience', 'aquarium', 'old', 'sydney', 'istanbul', 'shopping', 'dubai']
========================================
4
['falls', 'trail', 'hike', 'valley', 'yellowstone', 'memorial', 'water', 'road', 'area', 'views', 'right', 'walk', 'hiking', 'beach', 'grove', 'view', 'easy', 'visit', 'worth', 'ride']

# 34728870
0
['fun', 'train', 'free', 'place', 'food', 'time', 'vegas', 'experience', 'central', 'park', 'people', 'hk', 'old', 'lot', 'dubai', 'view', 'worth', 'singapore', 'amazing', 'way']
========================================
1
['museum', 'st', 'inside', 'art', 'place', 'jazz', 'gate', 'worth', 'time', 'museums', 'impressive', 'pm', 'temple', 'history', 'don', 'tiki', 'tour', 'work', 'west', 'world']
========================================
2
['tour', 'place', 'worth', 'time', 'museum', 'interesting', 'guide', 'building', 'walk', 'history', 'city', 'lovely', 'did', 'experience', 'inside', 'excellent', 'fun', 'little', 'way', 'trip']
========================================
3
['falls', 'trail', 'hike', 'valley', 'yellowstone', 'memorial', 'water', 'road', 'area', 'views', 'right', 'walk', 'hiking', 'beach', 'grove', 'view', 'easy', 'worth', 'ride', 'nature']
========================================
4
['diego', 'universal', 'san', 'parking', 'park', 'cars', 'time', 'rides', 'store', 'california', 'movie', 'center', 'tour', 'street', 'place', 'disney', 'river', 'shopping', 'spend', 'best']

# 2929292, max feature 400:


0
['museum', 'history', 'interesting', 'guide', 'building', 'hours', 'walk', 'worth', 'city', 'people', 'way', 'church', 'lovely', 'took', 'art', 'went', 'tickets', 'main', 'floor', 'little']
========================================
1
['worth', 'experience', 'fun', 'food', 'free', 'people', 'bar', 'trip', 'view', 'way', 'park', 'minutes', 'boat', 'went', 'bay', 'drinks', 'hotel', 'san', 'city', 'need']
========================================
2
['trail', 'falls', 'hike', 'road', 'memorial', 'water', 'area', 'views', 'bear', 'beach', 'walk', 'easy', 'morning', 'make', 'minutes', 'way', 'ride', 'view', 'parking', 'boat']
========================================
3
['town', 'worth', 'train', 'old', 'gardens', 'central', 'free', 'busy', 'walk', 'river', 'building', 'store', 'cars', 'area', 'best', 'friendly', 'lovely', 'views', 'bridge', 'walking']
========================================
4
['park', 'disney', 'parking', 'rides', 'love', 'food', 'kids', 'ride', 'movie', 'cars', 'pass', 'house', 'early', 'best', 'lots', 'fun', 'shows', 'husband', 'hour', 'area']

In [652]:
cluster0_mask = (cluster_df['cluster_k'] ==0)
cluster1_mask = (cluster_df['cluster_k'] ==1)
cluster2_mask = (cluster_df['cluster_k'] ==2)
cluster3_mask = (cluster_df['cluster_k'] ==3)
cluster4_mask = (cluster_df['cluster_k'] ==4)

# random state - 347837898


In [653]:
cluster_df[cluster0_mask]

,cluster_k,city_index
0,0,Amsterdam
3,0,Barcelona
5,0,Berlin
8,0,Budapest
12,0,Edinburgh
16,0,Istanbul
19,0,Kyoto
25,0,Madrid
28,0,Moscow
30,0,Nashville


In [647]:
cluster_df[cluster1_mask]

,cluster_k,city_index
2,1,Bangkok
6,1,Boston
7,1,Bridgetown
10,1,Chicago
11,1,Dubai
13,1,Helsinki
14,1,Hong Kong
18,1,Kuala Lumpur
21,1,Las Vegas
22,1,London


In [648]:
cluster_df[cluster2_mask]

,cluster_k,city_index
4,2,Bergen
15,2,Honolulu
33,2,Niagara Falls
54,2,Yellowstone National Park
55,2,Yosemite National Park


In [649]:
cluster_df[cluster3_mask]

,cluster_k,city_index
9,3,Chattanooga
17,3,Key West
24,3,Lucerne
29,3,Munich
34,3,Nice
40,3,Salzburg
52,3,Vienna


In [650]:
cluster_df[cluster4_mask]

,cluster_k,city_index
1,4,Anaheim
20,4,Lancaster
23,4,Los Angeles
35,4,Orlando


In [651]:
up_cluster_df = cluster_df[cluster2_mask]

In [399]:
up_cluster_df.columns = ['cluster_k', 'taObjectCity']

In [39]:
up_cluster_df.head()

,cluster_k,taObjectCity
7,1,Bridgetown
13,1,Helsinki
34,1,Nice
43,1,Seattle
46,1,Stockholm


In [40]:
cluster2_lst = list(cluster_df[cluster2_mask].city_index.values)

In [ ]:

for city in cluster2_lst:
    print(city)

In [ ]:
city_dict_df = pd.DataFrame(final_df.taObjectCity.unique(), columns = ['taObjectCity'])

In [ ]:
city_temp = city_dict_df.reset_index()

In [ ]:
city_df = city_temp.rename(columns = {'index':'city_id'})

In [ ]:
city_df.head(10)

In [ ]:
samp = [2,5,8]

In [ ]:
# for i in samp:
#     if i == city_df.city_id.values[i]:
#         print(i)
#         print(city_df.taObjectCity.values[i])

In [ ]:
for i in samp:
    row = city_df.loc[city_df['city_id'] == i]
    #mask = i == city_df.city_id.values[i]:
    #print(row.index)
    city = row.taObjectCity.values
    print (city[0])
    #print(type(row.taObjectCity.values))

In [ ]:
sampa = [(4.1000000000000005, 18), (4.153061224489796, 27), (4.399999999999999, 16), (4.198453608247422, 15), (4.315476190476191, 38), (4.006666666666667, 2), (5.0, 46), (3.740131578947369, 24), (4.346938775510204, 50), (4.491150442477878, 23), (4.818965517241379, 33), (4.31979695431472, 17), (4.353448275862069, 52), (34, 4.652061)]


In [ ]:
len(sampa)

In [ ]:
for i in city_df.city_id:
    if city_df.city_id == i:
        print(city_df.taObjectCity)

In [ ]:
`

In [ ]:
for city_id in pd.merge(up_cluster_df, city_df, how = 'left', on = 'taObjectCity').city_id:
    print (city_id)

In [ ]:
def matching_cities(cluster_df, cluster_num, city_dict):
    
    cluster2_mask = (cluster_df['cluster_k'] == cluster_num)
    cluster2_lst = list(cluster_df[cluster2_mask].city_index.values)
    
    selected_cities_idx = []
    for city in cluster2_lst:
        city_idx = city_dict[city]
        selected_cities_idx.append(city_idx)
        
        
    return selected_cities_idx
    
    
    
def cities_pred_rating_dict(selected_cities_idx):
    
    rating_dict = {}
    for city in selected_cities_idx:
        
        # call model
        # return overall_rating for each city
        
        rating_dict[city] = overall_rating
        
        
    return rating_dict



def top_three_city(rating_dict):
    
    tuple_lst = [(k, d[k]) for k in sorted(rating_dict, key=rating_dict.get, reverse=True)]
    
    top_three = tuple_lst[:3]
    
    return top_three
    
    
def print_rec_city(top_three, city_dict):
    
    top_three_city_with_rating = []
    
    for idx_1, rating in top_three.items():
        for city_name, idx_2 in city_dict.items():
            if idx_1 = idx_2
                top_three_city_with_rating.append((city_name, rating))
    
    for name, rating in top_three_city_with_rating:
        print(name, rating)
                
    

   cluster_k taObjectCity  city_id
0          1   Bridgetown       55
1          1     Helsinki       28
2          1         Nice       34
3          1      Seattle       41
4          1    Stockholm       49
5          1       Vienna        5

['visit', 'beautiful', 'worth', 'place', 'town', 'old', 'lovely', 'market', 'time', 'walk']

   cluster_k               taObjectCity  city_id
0          2                     Bergen       36
1          2                  Hong Kong       29
2          2                   Honolulu       21
3          2                    Lucerne       19
4          2  Yellowstone National Park       26
5          2     Yosemite National Park       53

['trail', 'hike', 'hk', 'train', 'valley', 'yellowstone', 'memorial', 'views', 'road', 'lake']